In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
# Assuming 'category_data' is your DataFrame containing data for a specific product and category_code
# Let's start with encoding categorical variables if needed


In [3]:
import warnings


In [4]:
warnings.filterwarnings('ignore')

In [5]:
# Load the datasets
oct_df = pd.read_csv('/data/CausalTrial/2019-Oct.csv')  # Update the path to your October dataset
nov_df = pd.read_csv('/data/CausalTrial/2019-Nov.csv')  # Update the path to your November dataset


In [6]:
# Combine the data
data = pd.concat([oct_df, nov_df])

In [7]:
del(oct_df)
del(nov_df)

In [8]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [9]:
# Step 1: Filter the DataFrame for 'electronics.smartphone' category and brand 'apple'
apple_smartphones_df = data[(data['category_code'] == 'electronics.smartphone') & (data['brand'] == 'apple')]


In [10]:
# Step 2: Identify price changes
# Sort the dataframe by event_time to ensure we are looking at prices in chronological order
apple_smartphones_df = apple_smartphones_df.sort_values(by='event_time')


In [11]:
# Calculate the difference in price between consecutive rows
apple_smartphones_df['price_diff'] = apple_smartphones_df['price'].diff()

In [12]:
# Step 3: Determine the first price decrease
# The first negative price_diff will be our intervention point
intervention_index = apple_smartphones_df[apple_smartphones_df['price_diff'] < 0].index[0]

In [13]:
intervention_index

20

In [14]:
# Set the intervention date as the date of the first price decrease
intervention_date = apple_smartphones_df.loc[intervention_index, 'event_time']

In [15]:
# Step 4: Calculate pre- and post-intervention prices and sales

# Pre-intervention prices and sales
pre_intervention_df = apple_smartphones_df[apple_smartphones_df['event_time'] < intervention_date]
pre_price_mean = pre_intervention_df['price'].mean()
pre_sales_count = pre_intervention_df[pre_intervention_df['event_type'] == 'purchase'].shape[0]

In [16]:
# Post-intervention prices and sales
post_intervention_df = apple_smartphones_df[apple_smartphones_df['event_time'] >= intervention_date]
post_price_mean = post_intervention_df['price'].mean()
post_sales_count = post_intervention_df[post_intervention_df['event_type'] == 'purchase'].shape[0]

In [17]:
# Display the results
print(f"Intervention Date: {intervention_date}")
print(f"Pre-Intervention Price Mean: {pre_price_mean}")
print(f"Post-Intervention Price Mean: {post_price_mean}")
print(f"Pre-Intervention Sales Count: {pre_sales_count}")
print(f"Post-Intervention Sales Count: {post_sales_count}")

Intervention Date: 2019-10-01 00:00:20 UTC
Pre-Intervention Price Mean: 1414.885
Post-Intervention Price Mean: 937.3154954074679
Pre-Intervention Sales Count: 0
Post-Intervention Sales Count: 242597


In [ ]:
## apple product has the price reduction on 2019-10-01 and sales has been increased by more than 2 lakhs product sold